# Анализ базы данных сервиса для чтения книг

**Целью исследования** является анализ базы данных для дальнейшего формирования ценностного предложения для нового продукта.

**Этапы исследования:**

1. Подготовка данных
2. Обзор данных
3. Исследовательский анализ данных]
4. Выводы

<a name="h1"></a>
### 1. Подготовка данных

#### Импорт библиотек

In [1]:
import pandas as pd
from sqlalchemy import create_engine

#### Установка параметров

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db']) 

#### Сохранение коннектора

In [3]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

<a name="h2"></a>
### 2. Обзор данных

Пропишем функцию для вывода первых строк таблиц

In [4]:
def show_sample(table_name):
    query = "SELECT * FROM {} LIMIT 3".format(table_name)
    return pd.io.sql.read_sql(query, con = engine)

Выведем первые строки всех таблиц

In [5]:
show_sample('books')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135


In [6]:
show_sample('authors')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie


In [7]:
show_sample('ratings')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5


In [8]:
show_sample('reviews')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...


In [9]:
show_sample('publishers')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books


<a name="h3"></a>
### 3. Исследовательский анализ данных

1. Определим количество книг, вышедших после 1 января 2000 года

In [10]:
query = "SELECT COUNT('book_id') AS number_of_books FROM books WHERE publication_date > '2000-01-01'"
pd.io.sql.read_sql(query, con = engine)

,number_of_books
0,819


2. Для каждой книги рассчитаем количество обзоров и среднюю оценку

In [15]:
query = """SELECT
                books.book_id AS book_id,
                books.title AS title,
                rating_reviews.avg_rating AS avg_rating,
                rating_reviews.cnt_reviews AS cnt_reviews
            FROM(SELECT 
                    ratings_1.book_id AS book_id, 
                    ratings_1.avg_rating AS avg_rating,
                    reviews_1.cnt_reviews AS cnt_reviews
                FROM 
                    (SELECT book_id, AVG(rating) AS avg_rating FROM ratings GROUP BY book_id) AS ratings_1 JOIN
                    (SELECT book_id, COUNT(review_id) AS cnt_reviews FROM reviews GROUP BY book_id) AS reviews_1 
                    ON ratings_1.book_id = reviews_1.book_id) AS rating_reviews JOIN 
                    books ON rating_reviews.book_id = books.book_id             
"""
        
pd.io.sql.read_sql(query, con = engine)

,book_id,title,avg_rating,cnt_reviews
0,1,'Salem's Lot,3.666667,2
1,2,1 000 Places to See Before You Die,2.500000,1
2,3,13 Little Blue Envelopes (Little Blue Envelope...,4.666667,3
3,4,1491: New Revelations of the Americas Before C...,4.500000,2
4,5,1776,4.000000,4
...,...,...,...,...
989,996,Wyrd Sisters (Discworld #6; Witches #2),3.666667,3
990,997,Xenocide (Ender's Saga #3),3.400000,3
991,998,Year of Wonders,3.200000,4
992,999,You Suck (A Love Story #2),4.500000,2


3. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [12]:
query = """SELECT  books.publisher_id, publishers.publisher, COUNT(books.book_id) AS number_of_books 
            FROM books 
            LEFT JOIN publishers ON publishers.publisher_id = books.publisher_id
            WHERE num_pages > 50 
            GROUP BY books.publisher_id, publishers.publisher_id 
            ORDER BY number_of_books DESC 
            LIMIT 1"""
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher,number_of_books
0,212,Penguin Books,42


4. Определим автора с самой высокой средней оценкой книг. Учитывать будем только книги с 50 и более оценками.

In [16]:
query = """SELECT
                authors.author AS author,
                authors_ratings.avg_rating AS avg_rating   
            FROM 
                (SELECT 
                    books.author_id AS author_id,
                    AVG(ratings_1.avg_rating) AS avg_rating                    
                FROM 
                    (SELECT book_id, AVG(rating) AS avg_rating
                    FROM ratings 
                    GROUP BY book_id 
                    HAVING COUNT(rating) > 50) AS ratings_1 
                     INNER JOIN
                     books ON ratings_1.book_id = books.book_id
                GROUP BY books.author_id) AS authors_ratings JOIN
                authors on authors.author_id = authors_ratings.author_id
                ORDER BY avg_rating DESC 
"""
        
pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


5. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [14]:
query = """   SELECT 
                    AVG(usernames_reviews.cnt_reviews) AS top_users_avg_reviews
                FROM 
                    (SELECT username FROM ratings GROUP BY username HAVING COUNT(rating_id) > 50) AS top_username JOIN 
                    (SELECT username, COUNT(review_id) AS cnt_reviews FROM reviews GROUP BY username) AS usernames_reviews
                    ON top_username.username = usernames_reviews.username"""
        
pd.io.sql.read_sql(query, con = engine)

,top_users_avg_reviews
0,24.333333


<a name="h4"></a>
### 4. Выводы

На приобретенном сервисе для чтения книг по подписке хранится 1 000 книг, в том числе 819 книг, выпущенных после 1 января 2000 года. На основании предоставленной информации для каждой книги были рассчитаны средний рейтинг и количество отзывов. Оба показателя варьируются от книги к книге. Наибольшее число книг, из представленных на сервисе, было выпущено британским издательством Penguin Books (42 книги), а наиболее популярным автором, согласно рейтингу читателей, является Дж. К. Роуллинг. Самые активные пользователи пишут в среднем около 24 обзоров на прочитанные книги.  
В целом можно сделать вывод, что на сервисе хранятся достаточно современные издания зарубежных авторов. Сервис пользуется популярностью среди читателей, особенно молодого возраста (до 35 лет), что подтверждается их активностью (оценки, обзоры) и предпочтениями (самый популярный автор - Дж. К. Роуллинг)